In [1]:
from medmnist import BloodMNIST
import matplotlib.pyplot as plt
from torchvision import models
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.models import VGG16_Weights
import cv2
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report


In [2]:
# Select device to accelerate the computation
#check if CUDA is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#check if MPS is available
device = torch.device("mps" if torch.backends.mps.is_available() else device)

print(device)

mps


In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

train_dataset = BloodMNIST(
    root='./datasets/', split='train', download=True, transform=transform)
val_dataset = BloodMNIST(root='./datasets/', split='val',
                         download=True, transform=transform)
test_dataset = BloodMNIST(root='./datasets/', split='test',
                          download=True, transform=transform)


# Data Loaders
batch_size = 50
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
# get sample data
sample, label = train_dataset[0]
print(f"Lable: {label}")
print(f"Image size: {sample.shape}")

Lable: [7]
Image size: torch.Size([3, 224, 224])


In [6]:
def get_vgg16_model():
    # Load pre-trained VGG16 model
    model = models.vgg16(weights=VGG16_Weights.DEFAULT)
    # Remove the existing classification head by replacing it
    model.classifier = nn.Sequential(
        nn.Linear(25088, 4096),  # First fully connected layer (unchanged)
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(4096, 256),  # Replace second FC layer with smaller one
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(256, 8),  # Binary classification (fracture vs. non-fracture)
        nn.Softmax(dim=1)  # Softmax for classification
    )
    return model

In [7]:
def to_rgb_image_np(gray_image_np):
    return np.dstack([gray_image_np, gray_image_np, gray_image_np])


def to_rgb_image_nps(gray_image_nps):
    for i in range(len(gray_image_nps)):
        gray_image_nps[i] = to_rgb_image_np(gray_image_nps[i])
    return gray_image_nps

In [8]:
def train(model, dataloader, optimizer=None, class_criterion=None):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(dataloader, desc="Training batch", leave=False):
        images = images.to(device)
        labels = labels.squeeze().long().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = class_criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)


def validate(model, dataloader, class_criterion=None):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Validation batch", leave=False):
            images = images.to(device)
            labels = labels.squeeze().long().to(device)

            outputs = model(images)
            loss = class_criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return running_loss / len(dataloader)

def test(model, dataloader):
    model.eval()
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Testing batch", leave=False):
            images = images.to(device)
            labels = labels.squeeze().long().to(device)

            outputs = model(images)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(torch.argmax(outputs, 1).cpu().numpy())

    # calculate accuracy, precision, recall, and F1 score
    print(classification_report(y_true, y_pred))
    

In [ ]:
# Train VGG16
# Use Cross-Entropy Loss and Adam optimizer.
model = get_vgg16_model()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 25
early_stop = 5
train_loss = []
val_loss = []
min_val_loss = np.inf
early_stop_counter = 0

# use tqdm for progress bar
for epoch in tqdm(range(epochs), desc="Epochs", leave=True):
    # Train for one epoch
    train_loss_epoch = train(
        model, train_loader, optimizer, criterion)
    train_loss.append(train_loss_epoch)

    # Validate on validation set
    val_loss_epoch = validate(model, val_loader, criterion)
    val_loss.append(val_loss_epoch)

    # Early stopping logic
    if val_loss_epoch < min_val_loss:
        min_val_loss = val_loss_epoch
        early_stop_counter = 0
        best_model = model.state_dict()
        # Save the model
        torch.save(best_model, "best_model_task_1.pth")
    else:
        early_stop_counter += 1

    if early_stop_counter >= early_stop:
        print(f"Early stopping at epoch {epoch}")
        break

# Plot the results
plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend()
plt.show()

Epochs:   0%|          | 0/25 [00:00<?, ?it/s]

Training batch:   0%|          | 0/240 [00:00<?, ?it/s]

Validation batch:   0%|          | 0/35 [00:00<?, ?it/s]

Training batch:   0%|          | 0/240 [00:00<?, ?it/s]

Validation batch:   0%|          | 0/35 [00:00<?, ?it/s]

Training batch:   0%|          | 0/240 [00:00<?, ?it/s]

Validation batch:   0%|          | 0/35 [00:00<?, ?it/s]

Training batch:   0%|          | 0/240 [00:00<?, ?it/s]

Validation batch:   0%|          | 0/35 [00:00<?, ?it/s]

In [ ]:
# test model
model.load_state_dict("best_model_task_1.pth")
model.eval()
test_loss, test_accuracy = validate(model, test_loader, criterion)

# calculate accuracy, precision, recall, and F1 score
test(model, test_loader)

In [ ]:
def Erosion(image):
    kernel = np.ones((3, 3), np.uint8)  # 3x3 结构元素
    return cv2.erode(image, kernel, iterations=1)

def Skeletonization(image):
    skel = np.zeros(image.shape, np.uint8)  # 骨架结果初始化
    ret, image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)  # 二值化
    element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))  # 交叉形结构元素
    done = False

    while not done:
        eroded = cv2.erode(image, element)  # 腐蚀
        temp = cv2.dilate(eroded, element)  # 先腐蚀再膨胀 (开运算)
        temp = cv2.subtract(image, temp)  # 提取去掉的部分
        skel = cv2.bitwise_or(skel, temp)  # 累加骨架
        image = eroded.copy()

        if cv2.countNonZero(image) == 0:  # 如果图像完全腐蚀完毕，停止
            done = True

    return skel

In [ ]:
class ErosionTransform:
    def __init__(self):
        pass

    def __call__(self, image):
        print(f"image type: {image.mode}")
        # 1. 将图像转换为灰度图像
        gray_image = image.convert("L")  # 'L'模式表示灰度图像

        # 2. 应用形态学操作 (例如膨胀操作)
        gray_image_np = np.array(gray_image)
        gray_image_np = cv2.dilate(
            gray_image_np, None, iterations=1)  # 膨胀操作，可根据需要调整

        # 3. 将灰度图像复制成3通道
        rgb_image_np = np.dstack(
            [gray_image_np, gray_image_np, gray_image_np])  # 复制单通道成3通道

        # 4. 将RGB图像转换为PIL图像
        rgb_image = Image.fromarray(rgb_image_np)

        return rgb_image
    
class SkeletonizationTransform:
    def __init__(self):
        pass

    def __call__(self, image):
        # 1. 将图像转换为灰度图像
        gray_image = image.convert("L")  # 'L'模式表示灰度图像

        # 2. 应用形态学操作 (例如骨架化操作)
        gray_image_np = np.array(gray_image)
        gray_image_np = Skeletonization(gray_image_np)
        
        # 3. 将灰度图像复制成3通道
        rgb_image_np = np.dstack(
            [gray_image_np, gray_image_np, gray_image_np])  # 复制单通道成3通道

        # 4. 将RGB图像转换为PIL图像
        rgb_image = Image.fromarray(rgb_image_np)

        return rgb_image

In [ ]:
trasnform = transforms.Compose([
    ErosionTransform(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

train_dataset = BloodMNIST(
    root='./datasets/', split='train', download=True, transform=transform)
val_dataset = BloodMNIST(root='./datasets/', split='val',
                         download=True, transform=transform)
test_dataset = BloodMNIST(root='./datasets/', split='test',
                          download=True, transform=transform)

In [ ]:
# Train VGG16
# Use Cross-Entropy Loss and Adam optimizer.
model = get_vgg16_model()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 25
early_stop = 5
train_loss = []
val_loss = []
min_val_loss = np.inf
early_stop_counter = 0

# use tqdm for progress bar
for epoch in tqdm(range(epochs), desc="Epochs", leave=True):
    # Train for one epoch
    train_loss_epoch = train(
        model, train_loader, optimizer, criterion)
    train_loss.append(train_loss_epoch)

    # Validate on validation set
    val_loss_epoch = validate(model, val_loader, criterion)
    val_loss.append(val_loss_epoch)

    # Early stopping logic
    if val_loss_epoch < min_val_loss:
        min_val_loss = val_loss_epoch
        early_stop_counter = 0
        best_model = model.state_dict()
        # Save the model
        torch.save(best_model, "best_model_task_2.pth")
    else:
        early_stop_counter += 1

    if early_stop_counter >= early_stop:
        print(f"Early stopping at epoch {epoch}")
        break

# Plot the results
plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend()
plt.show()

In [ ]:
# test model
model.load_state_dict("best_model_task_2.pth")
model.eval()
test_loss, test_accuracy = validate(model, test_loader, criterion)

# calculate accuracy, precision, recall, and F1 score
test(model, test_loader)

In [ ]:
trasnform = transforms.Compose([
    SkeletonizationTransform(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

train_dataset = BloodMNIST(
    root='./datasets/', split='train', download=True, transform=transform)
val_dataset = BloodMNIST(root='./datasets/', split='val',
                         download=True, transform=transform)
test_dataset = BloodMNIST(root='./datasets/', split='test',
                          download=True, transform=transform)

In [ ]:
# Train VGG16
# Use Cross-Entropy Loss and Adam optimizer.
model = get_vgg16_model()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 25
early_stop = 5
train_loss = []
val_loss = []
min_val_loss = np.inf
early_stop_counter = 0

# use tqdm for progress bar
for epoch in tqdm(range(epochs), desc="Epochs", leave=True):
    # Train for one epoch
    train_loss_epoch = train(
        model, train_loader, optimizer, criterion)
    train_loss.append(train_loss_epoch)

    # Validate on validation set
    val_loss_epoch = validate(model, val_loader, criterion)
    val_loss.append(val_loss_epoch)

    # Early stopping logic
    if val_loss_epoch < min_val_loss:
        min_val_loss = val_loss_epoch
        early_stop_counter = 0
        best_model = model.state_dict()
        # Save the model
        torch.save(best_model, "best_model_task_2.pth")
    else:
        early_stop_counter += 1

    if early_stop_counter >= early_stop:
        print(f"Early stopping at epoch {epoch}")
        break

# Plot the results
plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend()
plt.show()

In [ ]:
# test model
model.load_state_dict("best_model_task_2.pth")
model.eval()
test_loss, test_accuracy = validate(model, test_loader, criterion)

# calculate accuracy, precision, recall, and F1 score
test(model, test_loader)